## Name: Crystal Lau
## Date: 06/07/2018
## Challenge: Preparing a dataset for modeling

Assignment:
- select an outcome variable
- pick 4 or 5 other variables (one to two categorical, three to four continuous)
- explore variables using the univariate and bivariate methods
- create ten new features and explain reasoning behind each one
- use filtering methods to select the five best features and justify choices

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

df = pd.read_csv('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.csv')

In [ ]:
# preview of data
df.head()

### Data Exploration

In [ ]:
# columns and data types
df.dtypes

In [ ]:
# renaming columns into workable titles without all the newline nonsense
df.columns = ['City', 'Population', 'Violent_Crime', 'Murder_Nonnegligent_Manslaughter',
              'Rape_1', 'Rape_2', 'Robbery', 'Aggravated_Assault', 'Property_Crime', 'Burglary',
              'Larceny_Theft', 'Motor_Vehicle_Theft', 'Arson']

In [ ]:
# stripping commas
df['Population'] = df['Population'].str.replace(',','')
df['Violent_Crime'] = df['Violent_Crime'].str.replace(',','')
df['Rape_2'] = df['Rape_2'].str.replace(',','')
df['Robbery'] = df['Robbery'].str.replace(',','')
df['Aggravated_Assault'] = df['Aggravated_Assault'].str.replace(',','')
df['Property_Crime'] = df['Property_Crime'].str.replace(',','')
df['Burglary'] = df['Burglary'].str.replace(',','')
df['Larceny_Theft'] = df['Larceny_Theft'].str.replace(',','')
df['Motor_Vehicle_Theft'] = df['Motor_Vehicle_Theft'].str.replace(',','')

In [ ]:
df.head()

In [ ]:
# converting columns from objects to floats
df.astype({'Population':float, 'Violent_Crime':float, 'Rape_2':float, 'Robbery':float, 
           'Aggravated_Assault':float, 'Property_Crime':float, 'Burglary':float,
           'Larceny_Theft':float, 'Motor_Vehicle_Theft':float})

In [ ]:
# stripping whitespace
df.columns = df.columns.str.strip()

In [ ]:
(df.isnull().sum()/df.shape[0])*100

In [ ]:
df.drop(['Rape_1'], axis=1, inplace=True)

In [ ]:
df.fillna(0, inplace=True)

### Selecting variables

Categorical Variable:
- City

Continuous Variables:
- Population
- Violent_Crime
- Robbery
- Propety_Crime
- Larceny_Theft

In [ ]:
df[:10].drop(['City'], axis=1)

In [ ]:
# making scatterplot matrix of variables
sns.set_style('white')

g = sns.PairGrid(df.drop(['City'], axis=1), diag_sharey=False)
g.map_upper(plt.scatter, alpha=.5)
#g.map_lower(sns.regplot, scatter_kws=dict(alpha=0))
g.map_diag(sns.kdeplot, lw=3)
plt.show()

### Feature Engineering

- build models that capture the general trend of the data
- rather than using all the features, select parts of the columns that
- look at columns and see different ways to select features
- filter: looking at correlation btwn variable of interest and each column
- part of sci-kit learn library

### Filtering